## Akhilesh Pant (AU FTCA: MCA)

## Sentiment Analysis Using:
1. VADER (Valence Aware Dictionary and sEntiment Reasoner)
2. TextBlob
3. Flair (from the flair NLP library by Zalando)

## How the final sentiment is determined?
# The final sentiment score is calculated by combining:
1. 25% VADER Score
2. 25% TextBlob Score
3. 50% Flair Score (adjusted for polarity)
# The script then classifies sentiment as Positive, Neutral, or Negative based on the score.

In [ ]:
import re
import pickle
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import speech_recognition as sr
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob
from flair.models import TextClassifier
from flair.data import Sentence
import time

# Initialize sentiment analyzers
sia = SentimentIntensityAnalyzer()
flair_classifier = None  # Lazy loading for Flair
MODEL_PATH = "sentiment_model.pkl"

def get_flair_classifier():
    global flair_classifier
    if flair_classifier is None:
        try:
            flair_classifier = TextClassifier.load('sentiment')
        except Exception as e:
            print(f"Error loading Flair classifier: {e}")
            return None
    return flair_classifier

def clean_text(text):
    return re.sub(r'[^a-zA-Z\s]', '', text).lower()

def get_speech_input():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        recognizer.adjust_for_ambient_noise(source)
        try:
            audio = recognizer.listen(source, timeout=5, phrase_time_limit=10)
            return recognizer.recognize_google(audio)
        except sr.UnknownValueError:
            print("Could not understand audio")
        except sr.RequestError:
            print("Could not request results, check your internet connection")
        except sr.WaitTimeoutError:
            print("Listening timed out, please try again.")
    return None

def analyze_sentiment(text):
    cleaned_text = clean_text(text)
    vader_score = sia.polarity_scores(cleaned_text)['compound']
    blob_score = TextBlob(cleaned_text).sentiment.polarity
    flair_classifier = get_flair_classifier()
    flair_score, flair_label = 0, "NEUTRAL"
    if flair_classifier:
        sentence = Sentence(text)
        flair_classifier.predict(sentence)
        flair_sentiment = sentence.labels[0].to_dict()
        flair_score = flair_sentiment['confidence']
        flair_label = flair_sentiment['value']
    final_score = (0.25 * vader_score + 0.25 * blob_score + 0.50 * (flair_score if flair_label == 'POSITIVE' else -flair_score))
    sentiment = "Positive 😊" if final_score > 0.05 else "Negative 😠" if final_score < -0.05 else "Neutral 😐"
    return sentiment, final_score

def main():
    data = []
    while True:
        choice = input("(1) Text (2) Voice (3) Exit: ").strip()
        if choice == '1':
            text = input("Enter text: ")
        elif choice == '2':
            text = get_speech_input()
            if not text:
                continue
        elif choice == '3':
            break
        else:
            print("Invalid choice. Please try again.")
            continue

        start_time = time.time()
        sentiment, final_score = analyze_sentiment(text)
        data.append([text, sentiment, final_score])
        print(f"Text: {text}\nSentiment: {sentiment}\nScore: {final_score:.4f}\n⏳ Analysis Time: {time.time() - start_time:.2f}s\n")

    if data:
        df = pd.DataFrame(data, columns=["Text", "Sentiment", "Final Score"])
        print(df)

if __name__ == "__main__":
    main()

(1) Text (2) Voice (3) Exit:  2


Listening timed out, please try again.
